<a href="https://colab.research.google.com/github/bokutachi256/gisday2025/blob/main/GISDAY2025_Python%E3%81%A8GIS%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%81%AB%E3%82%88%E3%82%8B%E5%9C%B0%E5%BD%A2%E5%88%86%E6%9E%90%E5%85%A5%E9%96%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PythonとGISを使った機械学習による地形分析入門

* GIS Day in 東京 2025 Eコース
* Python とGISを使った機械学習による地形分析入門
* 東京都立大学 都市環境学部 地理環境学科 中山大地（daichi@tmu.ac.jp）
* 2025年11月22日 東京都立大学 南大沢キャンパス

* github repository: [www.github.com/bokutachi256/gisday2025](https://www.github.com/bokutachi256/gisday2025)
* Google ColaboratoryのURL [https://colab.research.google.com/](https://colab.research.google.com/)
* 参考になるページ
  - [scikit-learn のマニュアルページ](https://gdal.org/python/)
  - [平成29年7月九州北部豪雨に関する情報（国土地理院）](https://www.gsi.go.jp/BOUSAI/H29hukuoka_ooita-heavyrain.html)
  - [正射画像判読図（朝倉・東峰地区）・GeoJSONファイルのダウンロード（上記ページ）](https://www.gsi.go.jp/BOUSAI/H29hukuoka_ooita-heavyrain.html#9)
* 更新履歴
  * 2025年11月22日：初版公開


## はじめに

この講習では，国土地理院が公開している平成29年7月九州北部豪雨時の土砂崩壊ポリゴンと10m解像度DEMから，
機械学習の一種である決定木を用いて土砂崩壊地を推定するモデルを作成することを目的としています．
手順の概要は以下になります．

1. 平成29年7月九州北部豪雨の災害状況ベクタデータを読み込み，土砂崩壊ポリゴンを抽出する．
2. あらかじめDEMから計算した地形量ラスタデータを読み込む．
3. 対象地域を覆う100mメッシュごとに地形量と土砂崩壊の有無を集計する．
4. 100mメッシュごとの地形量を説明変数，土砂崩壊の有無を目的変数とする決定木を作成する．
5. 作成した決定木を評価し，チューニングする．
6. 作成した決定木に基づく混同行列マップから土砂崩壊ポテンシャルを求めて地図化する．
7. 作成した決定木に基づく葉ノードを地図化し，ノードごとの土砂崩壊の状態を考察する．
8. メッシュに集計した結果をエクスポートし，GISで使えるようにする．

## 使い方

### 動作環境について

1. Googleドライブに`gisday2025`フォルダを作成する（フォルダ名は「すべて小文字」にすること）．
2. 共有フォルダ（[https://drive.google.com/drive/folders/17yH9qPEzIxRaS94WipiH3bKX5r1aCa9Z?usp=sharing](https://drive.google.com/drive/folders/17yH9qPEzIxRaS94WipiH3bKX5r1aCa9Z?usp=sharing)）から以下のデータをダウンロードしてください．
   * `20170810asakura_toho_handokuzu.geojson`：国土地理院からダウンロードした平成29年7月九州北部豪雨時の朝倉・東峰地区の正射画像判読データ
   * `AOI.geojson`：集計用の100mメッシュポリゴン
   * `output_dem_reprojected.tif`：10m 解像度数値標高モデル
   * `output_dis.tif`：侵食高
   * `output_hillshade.tif`：陰影起伏図
   * `output_plc.tif`：平面曲率（PlC）
   * `output_prc.tif`：断面曲率（PrC）
   * `output_slope_deg.tif`：傾斜量
   * `output_stilog.tif`：土砂移動指数（STI）
   * `output_wetnessindex.tif`：湿潤指数（WI）
3. Google Driveに作成した`gisday2025`フォルダに上記のファイルをアップロードしてください．


### 実行環境の自動判定

このプログラムがgoogle colaboratoryで動いているか，
ローカルなPythonで動いているか判別し，
GoogleDriveのマウントやベースディレクトリの指定を自動的に行います．

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
if 'google.colab' in sys.modules:
    # 必要なライブラリをインストール
    %pip install rasterio
    %pip install rasterstats
    %pip install geocube
    %pip install mapclassify

    # GoogleDriveのマウント
    from google.colab import drive
    drive.mount('/content/drive')
    # GoogleDrive内の"マイドライブ/gisday2025/"フォルダにアクセスできるような設定を行う．
    # フォルダ名の最後に`/`を必ず追加すること．

    # ベースディレクトリの指定
    base_dir = "/content/drive/My Drive/gisday2025/"

else:
    # ローカルPCを使う場合は./data/フォルダをマウントする
    # フォルダ名の最後に`/`を必ず追加すること．
    base_dir = "./data/"

### ライブラリの読み込み

使用するライブラリを説明します．

* `numpy`：多次元配列計算のライブラリ
* `pandas`: データベース的な処理を行うライブラリ
* `geopandas`：地理情報を扱うライブラリ
* `folium`：インタラクティブマップを作成するライブラリ
* `rasterio`：ラスタ型の空間データを扱うライブラリ
* `geocube`：ベクタ型データとラスタ型データを変換するライブラリ
* `rasterstats`：ラスタデータの集計（ゾーン集計）を行うライブラリ
* `matplotlib`：画像表示ライブラリ
* `sklearn`：機械学習用ライブラリ
* `google.colab`: Google Colaboratory用ライブラリ．google driveのマウントに使用．

以下で使用するライブラリを導入します．

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import rasterio
from geocube.api.core import make_geocube
from rasterstats import zonal_stats
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.model_selection import train_test_split, \
    GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree,\
    export_graphviz
from sklearn.metrics import accuracy_score, classification_report,\
    confusion_matrix

## データの読み込みと確認

### 被害状況ベクタデータの読み込み

使用する被害状況ベクタデータ（平成29年7月九州北部豪雨時の朝倉・東峰地区の正射画像判読データ）と
集計用の100mメッシュデータのGeoJSONを読み込みます．
読み込んだデータはGeoPandasのGeoDataFrame型式になります．
読み込んだデータは`to_crs`メソッドを用いて世界測地系（JGD2011）の平面直角座標系（公共測量座標系）第Ⅱ系（epsg6670）に投影変換します．
読み込んだ被害状況ベクタデータは`ls_polygon`，
メッシュデータは`aoi_polygon`とします．

In [ ]:
# 被害状況ベクタデータのGeoJSONを読み込み，
# epsg6670（JGD2011平面直角座標系第2系）に変換する

ls_polygon = gpd.read_file(base_dir + "20170810asakura_toho_handokuzu.geojson")\
    .to_crs(epsg=6670)

# 分析対象のメッシュを読み込み，epsg6670に変換する
# （もともとepsg6670なので変換は不要だが念のため明示しておく）

aoi_polygon = gpd.read_file(base_dir + "AOI.geojson")\
    .to_crs(epsg=6670)

GeoDataFrameの`plot`メソッドを使って，被害状況ベクタデータを表示してみましょう．

In [ ]:
ls_polygon.plot()

確認のため最後の5行を表示してみると，判読範囲や判読不能範囲などのデータが含まれていることがわかります．

In [ ]:
ls_polygon.tail(5)

被害状況ベクタデータから必要なデータのみを取り出します．
とりあえずポリゴンのみを抽出しましょう．
`ls_polygon`の`geometry.type`プロパティが`Polygon`になっているのがポリゴンデータです．


In [ ]:
ls_polygon = ls_polygon[ls_polygon.geometry.type=='Polygon']

ポリゴンのみ取り出せているか表示して確認します．

In [ ]:
ls_polygon.plot()

表示されたポリゴンの属性を表示します．
`ls_polygon`の`name`属性にポリゴンの種類が入ってるので，`unique`メソッドを使って値を確認します．

In [ ]:
ls_polygon.name.unique()

洪水流到達範囲，土砂崩壊地，判読不能範囲の3種類がありました．
このうち土砂崩壊地のみ表示してみます．

In [ ]:
ls_polygon[ls_polygon.name=='土砂崩壊地'].plot()

集計用のポリゴンも表示してみます．
集計用のポリゴンもGeoDataFrameになっていますので，`explore`メソッドを使ってインタラクティブマップに表示します．

In [ ]:
aoi_polygon.explore()

この集計用メッシュごとに地形量と土砂崩壊地の有無を集計します．

### 地形データの読み込みと確認

`rasterio`を使ってDEMを読み込みます．
本資料で使用するDEMは[基盤地図情報](https://www.gsi.go.jp/kiban/)の数値標高モデル（DEM10B）を使用しています．
xmlをダウンロードしてGIS Day in 東京 2019のEコース「Pythonを用いたDEM処理」のプログラム[「1_基盤地図情報の読み込み.ipynb」](https://github.com/bokutachi256/gisday2019/blob/main/1_%E5%9F%BA%E7%9B%A4%E5%9C%B0%E5%9B%B3%E6%83%85%E5%A0%B1%E3%81%AE%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF.ipynb)を使用してGeoTiffに変換しています．
ダウンロードしたDEMのメッシュは以下です．

* 503005
* 503006
* 503007
* 503015
* 503016
* 503017
* 503025
* 503026
* 503027
* 503100
* 503110
* 503120

GeoTiffに変換したDEMは世界測地系（JGD2011）の平面直角座標系（公共測量座標系）第Ⅱ系（epsg6670）に変換し，
10 m解像度にリサンプリングしました．


rasterioを用いてDEMを読み込みます．

In [ ]:
# DEMの読み込み
dem_data = rasterio.open(base_dir + "output_dem_reprojected.tif")

確認のため読み込んだDEMを表示してみます．
rasterioは多バンドデータに対応しているので，読み込んだDEMのバンドを指定する必要があります．
バンドは一つしかないので，`dem_data.read(1)`を`imshow`で表示します．


In [ ]:
# DEMの表示
fig, ax = plt.subplots()
img = ax.imshow(dem_data.read(1), vmin=0, vmax=1000, cmap='terrain')
plt.title('DEM (Digital Elevation Model)')
plt.colorbar(img, ax=ax)
plt.show()

DEM以外の地形量も読み込みます．
読み込む地形量は以下のもので，こちらについてもGIS Day in 東京 2019 Eコース「Pythonを用いたDEM処理」のプログラムを用いて処理しています．

* 傾斜量：`output_slope_deg.tif`：[（2_傾斜量と曲率の計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/2_%E5%82%BE%E6%96%9C%E9%87%8F%E3%81%A8%E6%9B%B2%E7%8E%87%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)
* 平面曲率（PlC）：`output_plc.tif`[（2_傾斜量と曲率の計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/2_%E5%82%BE%E6%96%9C%E9%87%8F%E3%81%A8%E6%9B%B2%E7%8E%87%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)
* 断面曲率（PrC）：`output_prc.tif`[（2_傾斜量と曲率の計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/2_%E5%82%BE%E6%96%9C%E9%87%8F%E3%81%A8%E6%9B%B2%E7%8E%87%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)
* 土砂移動指数（STI）：`output_stilog.tif`：[（4_Wetness_indexとSTIの計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/4_Wetness_index%E3%81%A8STI%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)
* 湿潤指数（WI）：`output_wetnessindex.tif`：[（4_Wetness_indexとSTIの計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/4_Wetness_index%E3%81%A8STI%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)
* 侵食高：`output_dis.tif`：[（3_接峰面と陰影図の計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/3_%E6%8E%A5%E5%B3%B0%E9%9D%A2%E3%81%A8%E9%99%B0%E5%BD%B1%E5%9B%B3%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)
* 陰影起伏図：`output_hillshade.tif`：[（3_接峰面と陰影図の計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/3_%E6%8E%A5%E5%B3%B0%E9%9D%A2%E3%81%A8%E9%99%B0%E5%BD%B1%E5%9B%B3%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)

侵食髙については[（3_接峰面と陰影図の計算.ipynb）](https://github.com/bokutachi256/gisday2019/blob/main/3_%E6%8E%A5%E5%B3%B0%E9%9D%A2%E3%81%A8%E9%99%B0%E5%BD%B1%E5%9B%B3%E3%81%AE%E8%A8%88%E7%AE%97.ipynb)で求めた接峰面（`output_summit.tif`）とDEM（`output_dem_reprojected.tif`）の差分です．
差分はArcGIS ProやQGISなどのデスクトップ型GISで求められます．
もちろんプログラムを改造しても構いません．

In [ ]:
# 傾斜量，平面曲率（PlC），断面曲率（PrC），土砂移動指数（STI），
# 湿潤指数（WI），侵食高（dis），陰影起伏図（hillshade）の読み込み

slope_data = rasterio.open(base_dir + "output_slope_deg.tif")
plc_data = rasterio.open(base_dir + "output_plc.tif")
prc_data = rasterio.open(base_dir + "output_prc.tif")
sti_data = rasterio.open(base_dir + "output_stilog.tif")
wi_data= rasterio.open(base_dir + "output_wetnessindex.tif")
dis_data= rasterio.open(base_dir + "output_dis.tif")
hillshade_data= rasterio.open(base_dir + "output_hillshade.tif")

確認のため傾斜量を表示してみます．

In [ ]:
# 傾斜量の表示
fig, ax = plt.subplots()
img = ax.imshow(slope_data.read(1), vmin=0, vmax=45, cmap='Reds')
ax.set_title('Slope angle (deg)')
plt.colorbar(img, ax=ax)
plt.show()

傾斜量の一部を拡大して表示してみます．

In [ ]:
# 傾斜量の一部を拡大して表示

fig, ax = plt.subplots(figsize=(10, 10))
img = ax.imshow(slope_data.read(1)[2000:2500, 2000:2500], vmin=0, vmax=45, cmap='Reds')
plt.colorbar(img)
plt.title('Slope angle (deg)')
plt.show()

すべての地形データを一覧表示してみましょう．

In [ ]:
# すべての地形データを一度に表示
fig = plt.figure(figsize=(15, 15))
ax1 = fig.add_subplot(3, 3, 1)
ax2 = fig.add_subplot(3, 3, 2)
ax3 = fig.add_subplot(3, 3, 3)
ax4 = fig.add_subplot(3, 3, 4)
ax5 = fig.add_subplot(3, 3, 5)
ax6 = fig.add_subplot(3, 3, 6)
ax7 = fig.add_subplot(3, 3, 7)
ax8 = fig.add_subplot(3, 3, 8)

# DEMの表示
img = ax1.imshow(dem_data.read(1)[2000:2500, 2000:2500], vmin=0,
                vmax=1000, cmap='terrain')
plt.colorbar(img, ax=ax1)
ax1.set_title('DEM')

# 陰影図の表示
img = ax2.imshow(hillshade_data.read(1)[2000:2500, 2000:2500], vmin=0,
                vmax=255, cmap='gray')
plt.colorbar(img, ax=ax2)
ax2.set_title('Hillshade')

# 開析髙の表示
img = ax3.imshow(dis_data.read(1)[2000:2500, 2000:2500], vmin=0,
                vmax=100, cmap='Purples')
plt.colorbar(img, ax=ax3)
ax3.set_title('Dissection Height (m)')

# 傾斜量の表示
img = ax4.imshow(slope_data.read(1)[2000:2500, 2000:2500], vmin=0,
                vmax=45, cmap='Reds')
plt.colorbar(img, ax=ax4)
ax4.set_title('Slope angle (deg)')

# 平面曲率（PlC）の表示
img = ax5.imshow(plc_data.read(1)[2000:2500, 2000:2500], vmin=-0.05,
                vmax=0.05, cmap='bwr')
plt.colorbar(img, ax=ax5)
ax5.set_title('Plan Curvature (PlC)')

# 断面曲率（PrC）の表示
img = ax6.imshow(prc_data.read(1)[2000:2500, 2000:2500], vmin=-0.05,
                vmax=0.05, cmap='bwr')
plt.colorbar(img, ax=ax6)
ax6.set_title('Profile Curvature (PrC)')

# 土砂移動指数（STI）の表示
img = ax7.imshow(sti_data.read(1)[2000:2500, 2000:2500], vmin=0,
                vmax=5, cmap='Greens')
plt.colorbar(img, ax=ax7)
ax7.set_title('Sediment Transport Index (STI)')

# 湿潤指数（WI）の表示
img = ax8.imshow(wi_data.read(1)[2000:2500, 2000:2500], vmin=0,
                vmax=20, cmap='Blues')
plt.colorbar(img, ax=ax8)
ax8.set_title('Wetness Index (WI)')

plt.show()

## 説明変数の準備

### 地形特徴量（連続量）の集計

`zonal_stats`を使って集計メッシュごとに傾斜量を集計します．
傾斜量は連続量のため，メッシュごとの最小値（`min`），最大値（`max`），平均値（`mean`），標準偏差（`std`），値域（`range`）を集計します．
`zonal_stats`の引数は集計単位のポリゴン（`aoi_polyton`），集計するラスタデータ（`slope_data.read(1)`），
集計するラスタデータのアフィンパラメーター（`slope_data.transform`），集計する統計量です．

ここでは傾斜量を集計したいので，`rasterio`で読み込んだ傾斜量`slope_data`を指定します．
`rasterio`で読み込んだラスタデータは多バンドになるので，どのバンドを集計するか指定する必要があります．
傾斜量は1バンドしかないので`slope_data.read(1)`とします．
ラスタデータのアフィンパラメーターは`transform`プロパティで取得できるので，`slope_data.transform`とします．
集計する統計量はリストで指定します．


In [ ]:
# 傾斜量の集計
stats = zonal_stats(aoi_polygon, slope_data.read(1),
                    affine=slope_data.transform,
                    stats=['min', 'max', 'mean', 'std', 'range'])


集計結果は`stats`に入るので，これをPandasのDataFrameに変換します．
DataFrameの列名は`min`, `max`, `mean`, `std`, `range`になりますが，
この後で他の地形特徴量を集計した時に列名が同じになってしまうので
傾斜量の集計結果だということがわかるように列名を変更（列名に`slope_`を加える）します．
`rename`プロパティで`inplace=True`を指定して，`slope_df`自体の列名を変更します．

In [ ]:
# 傾斜量の集計結果をdataframeに変換
slope_df = pd.DataFrame(stats)
# 列名の変更
slope_df.rename(columns={'min': 'slope_min', 'max': 'slope_max',
                        'mean': 'slope_mean', 'std': 'slope_std',
                        'range': 'slope_range'}, inplace=True)

集計結果を確認します．

In [ ]:
print(slope_df)

ちゃんと集計されているようです．

次に傾斜量以外の地形量も集計しましょう．
陰影起伏は地形の概観を見るためだけに使いますので，集計の対象からは外します．
曲率（`plc_data`と`prc_data`）は後ほど斜面形状に変換して集計するのでこちらも集計から外します．

In [ ]:
#  標高の集計
stats = zonal_stats(aoi_polygon, dem_data.read(1),
                    affine=dem_data.transform,
                    stats=['min', 'max', 'mean', 'std', 'range'])
dem_df = pd.DataFrame(stats)
dem_df.rename(columns={'min': 'alt_min', 'max': 'alt_max',
                    'mean': 'alt_mean', 'std': 'alt_std',
                    'range': 'alt_range'}, inplace=True)

#  侵食高の集計
stats = zonal_stats(aoi_polygon, dis_data.read(1),
                    affine=dis_data.transform,
                    stats=['min', 'max', 'mean', 'std', 'range'])
dis_df = pd.DataFrame(stats)
dis_df.rename(columns={'min': 'dis_min', 'max': 'dis_max',
                    'mean': 'dis_mean', 'std': 'dis_std',
                    'range': 'dis_range'}, inplace=True)

# STIの集計
stats = zonal_stats(aoi_polygon, sti_data.read(1),
                    affine=sti_data.transform,
                    stats=['min', 'max', 'mean', 'std', 'range'])
sti_df = pd.DataFrame(stats)
sti_df.rename(columns={'min': 'sti_min', 'max': 'sti_max',
                    'mean': 'sti_mean', 'std': 'sti_std',
                    'range': 'sti_range'}, inplace=True)

# WIの集計
stats = zonal_stats(aoi_polygon, wi_data.read(1),
                    affine=wi_data.transform,
                    stats=['min', 'max', 'mean', 'std', 'range'])
wi_df = pd.DataFrame(stats)
wi_df.rename(columns={'min': 'wi_min', 'max': 'wi_max',
                    'mean': 'wi_mean', 'std': 'wi_std',
                    'range': 'wi_range'}, inplace=True)

集計したデータを確認してみましょう．
ここでは侵食髙`dis_df`を表示しますが，他の地形量についても確認してみましょう．

In [ ]:
print(dis_df)

集計した地形量をメッシュに結合します．
集計した地形データの数はメッシュと同じなので，`pd.concat`を使ってメッシュデータに横結合します．

結合したいDataFrameをリストで与え，`axis=1`として横結合を指定します．


In [ ]:
# 集計データの結合
aoi_agg_df = pd.concat([aoi_polygon, dem_df, slope_df, dis_df, sti_df, wi_df], axis=1)

### カテゴリデータ（斜面形状）の集計

### 曲率から斜面形状への変換

平面曲率（`plc_data`）と断面曲率（`prc_data`）も集計します．
曲率は地形の平面曲率（Plan Curvature, PlC）つまり地形の水平断面（等高線）の曲率と，地形の垂直断面の曲率（Profile Curvature; PrC）を示しています．
平面曲率が負の場合は谷型斜面（等高線が凹），0は等高線が直線，正は尾根型斜面（等高線が凸）で，
断面曲率が負の場合は凸型斜面，0は直線斜面，正は凹型斜面となります．

メッシュごとに平面曲率を集計した場合，例えばあるメッシュの尾根型斜面と谷型斜面の面積が50%ずつだった場合，
平均を取ってしまうと0になってしまい，そのメッシュの平均的な斜面形状は直線型ということになってしまいます．
このため，曲率の単純な統計量にはあまり意味がありません．
これを避けるために，平面曲率と断面曲率から斜面形状を表すカテゴリ（名義尺度）データを作成し，
メッシュ内でそれぞれの斜面形状の面積比を求めます．

それでは平面曲率から斜面形状（谷型斜面，直線斜面，尾根型斜面）を求めましょう．
平面曲率が負の場合は谷型斜面になりますが，絶対値が小さい場合はほとんど直線斜面とみなしてよいでしょう．
このため平面曲率が-0.005未満を谷型斜面，-0.005以上0.005未満を直線斜面，0.005以上を尾根型斜面とします．

`np.where`を使って平面曲率を斜面形状に再分類します．
再分類結果は`plc_shape`とし，1を谷型斜面，2を直線斜面，3を尾根型斜面とします．

In [ ]:
# 平面曲率（PlC）を3種類の形状に分類

# -0.005未満を谷型斜面（ラベル1），-0.005〜0.005を直線斜面（ラベル2）
# 0.005以上を尾根型斜面（ラベル3）として分類

plc_shape = np.where(plc_data.read(1) < -0.005, 1, 0)
plc_shape = np.where((plc_data.read(1) >= -0.005) & (plc_data.read(1) < 0.005)\
                    & (plc_shape == 0), 2, plc_shape)
plc_shape = np.where((plc_data.read(1) >= 0.005) & (plc_shape == 0), 3, plc_shape)

分類した斜面形状を表示してみます．

In [ ]:
# 分類した斜面形状を表示

img = plt.imshow(plc_shape[2000:2200, 2000:2200], cmap='rainbow')
plt.colorbar(img)

同様に断面曲率も斜面形状に再分類します．
-0.005未満を凸型斜面，-0.005〜0.005を直線斜面，0.005以上を凹型斜面とし，
それぞれ1，2，3とします．
分類した結果も表示します．

In [ ]:
# 断面曲率（PrC）を3種類の形状に分類

# -0.005未満を凸斜面（ラベル1），-0.005〜0.005を直線斜面（ラベル2），
# 0.005以上を凹斜面（ラベル3）として分類
prc_shape = np.where(prc_data.read(1)<-0.005, 1, 0)
prc_shape = np.where((prc_data.read(1)>=-0.005) & (prc_data.read(1)<0.005)
                    & (prc_shape==0), 2, prc_shape)
prc_shape = np.where((prc_data.read(1)>=0.005) & (prc_shape==0), 3, prc_shape)

# 分類した斜面形状を表示
img = plt.imshow(prc_shape[2000:2200, 2000:2200], cmap='rainbow')
plt.colorbar(img)


作成した斜面形状データを集計しましょう．
斜面形状はカテゴリ（質的）データなので，斜面形状ごとのピクセル数をカウントします．
集計には量的データと同様に`zonal_stats`を使いますが，
カテゴリデータの場合は`category=True`を指定します．
また，数値とカテゴリの対応表をディクショナリで与えます．
ここでは対応表を`plc_catmat`とし，`plc_catmap = {1: 'plc_valley', 2: 'plc_linear', 3: 'plc_ridge'}`とします．
集計結果を`plc_cat_df`に格納し，`astype`メソッドを使って実数に変換します．

In [ ]:
# 谷型斜面（ラベル1），直線斜面（ラベル2），尾根型斜面（ラベル3）
plc_catmap = {1: 'plc_valley', 2: 'plc_linear', 3: 'plc_ridge'}
stats = zonal_stats(aoi_polygon, plc_shape, affine=plc_data.transform,
                    categorical=True, category_map=plc_catmap)
plc_cat_df = pd.DataFrame(stats).astype(float)


確認のため集計結果を表示します．

In [ ]:
print(plc_cat_df)

`plc_valley`，`plc_linear`，`plc_ridge`の列にそれぞれ谷型斜面，直線斜面，尾根型斜面のピクセル数が入ります．
`0`の列はいずれにも分類されなかったピクセル数です．

次にそれぞれの斜面形状の割合を求めます．
割合は`plc_valley_ratio`，`plc_linear_ratio`，`plc_ridge_ratio`に格納します．
欠損値は`nan`になるので，`fillna`メソッドを使って0に置き換えます．

In [ ]:
# 各斜面形状の面積比を計算
plc_cat_df['plc_valley_ratio'] = plc_cat_df['plc_valley'] /\
    sum(plc_cat_df.loc[0, ['plc_valley', 'plc_linear', 'plc_ridge']])
plc_cat_df['plc_linear_ratio'] = plc_cat_df['plc_linear'] /\
    sum(plc_cat_df.loc[0, ['plc_valley', 'plc_linear', 'plc_ridge']])
plc_cat_df['plc_ridge_ratio'] = plc_cat_df['plc_ridge'] /\
    sum(plc_cat_df.loc[0, ['plc_valley', 'plc_linear', 'plc_ridge']])

# 欠損値を0で埋める
plc_cat_df = plc_cat_df.fillna(0.0)

求めた面積比を表示してみましょう．

In [ ]:
print(plc_cat_df)

斜面形状の面積比をメッシュに結合します．
必要なのは面積比（`plc_valley_ratio`，`plc_linear_ratio`，`plc_ridge_ratio`）のみなので，
これらを`concat`を使って集計結果のメッシュデータ`aoi_agg_df`に横結合します．

In [ ]:
# 斜面形状の面積比をメッシュに結合する

aoi_agg_df = pd.concat([aoi_agg_df,
                        plc_cat_df[['plc_valley_ratio', 'plc_linear_ratio',\
                                    'plc_ridge_ratio']]],
                        axis=1)


断面曲率も斜面形状ごとの面積比を求め，集計メッシュデータに横結合します．

In [ ]:
# 断面曲率も斜面形状# （凸斜面（ラベル1），直線斜面（ラベル2），凹斜面（ラベル3））に変換して面積率を集計する

# 凸斜面（ラベル1），直線斜面（ラベル2），凹斜面（ラベル3）
prc_catmap = {1: 'prc_convex', 2: 'prc_linear', 3: 'prc_concave'}
stats = zonal_stats(aoi_polygon, prc_shape, affine=prc_data.transform,
                    categorical=True, category_map=prc_catmap)
prc_cat_df = pd.DataFrame(stats).astype(float)

# 各斜面形状の面積比を計算
prc_cat_df['prc_convex_ratio'] = prc_cat_df['prc_convex'] /\
    sum(prc_cat_df.loc[0, ['prc_convex', 'prc_linear', 'prc_concave']])
prc_cat_df['prc_linear_ratio'] = prc_cat_df['prc_linear'] /\
    sum(prc_cat_df.loc[0, ['prc_convex', 'prc_linear', 'prc_concave']])
prc_cat_df['prc_concave_ratio'] = prc_cat_df['prc_concave'] /\
    sum(prc_cat_df.loc[0, ['prc_convex', 'prc_linear', 'prc_concave']])

# 欠損値を0で埋める
prc_cat_df = prc_cat_df.fillna(0.0)

# 斜面形状の面積比をメッシュに結合する
aoi_agg_df = pd.concat([aoi_agg_df,
                        prc_cat_df[['prc_convex_ratio', 'prc_linear_ratio',\
                                    'prc_concave_ratio']]],
                        axis=1)


結合したデータをマップ表示して確認しましょう．
ここでは例として標高の値域（`alt_range`）を表示します．
ほかの値も表示してみてください．

In [ ]:
aoi_agg_df.plot(column='alt_range', legend=True)

## 目的変数の準備

この資料では，土砂崩壊の有無を目的変数として判別モデルを作成します．
土砂崩壊はベクタデータとして与えられており，
これをラスタ型データに変換した後にメッシュごとに面積比を集計し，
メッシュ中の一定面積以上に土砂災害のあるメッシュを「土砂崩壊あり」のメッシュとします．

### 目的変数を集計する

被害状況ベクタデータからポリゴンのみを抽出したデータ`ls_polygon`は`name`列が災害の種別を表しています．
これらのポリゴンをラスタ型に変換するために，災害の種別のラベルを整数型で与えます．
具体的には「土砂崩壊地」を`1`，「洪水流到達範囲」を`2`，これ以外のポリゴンは`0`にします．
ラベルは新しい列`cat`に格納します．

In [ ]:
print(ls_polygon.head(3))

In [ ]:
# 教師データにラベルを加える
# 土砂崩壊地ポリゴンのcatを1，洪水流到達範囲ポリゴンのcatを2に設定
ls_polygon['cat'] = 0
ls_polygon.loc[ls_polygon['name']=='土砂崩壊地', 'cat'] = 1
ls_polygon.loc[ls_polygon['name']=='洪水流到達範囲', 'cat'] = 2

次にラベル付けした被害状況ベクタデータをラスタデータに変換します．
変換したラスタデータの座標系・測地系はDEMなどと同じ平面直角座標系第Ⅱ系・JGD2011（epsg6670）で，解像度も同じ10mにします．

ラスタへの変換は`make_geocube`を使います．
`make_geocube`の引数としては以下になります．
変換するベクタデータを`vector_data=ls_polygon`，
変換後のラスタデータのCRSを`output_crs='epsg:6670'`，
ラスタに埋め込む値のある列を`measurements=['cat']`，
ラスタの解像度を`resolution=(-10, 10)`，
`fill=0`でポリゴンのない場所の値を0で埋めます．

解像度の指定ですが，南北方向はマイナスをつける必要があります．

In [ ]:
# resolutionのy方向はマイナスにする
ls_grid = make_geocube(
    vector_data=ls_polygon,
    output_crs='epsg:6670',
    measurements=['cat'],
    resolution=(-10, 10),
    fill=0,
)

作成したラスタデータは`ls_grid`に格納します．
`ls_grid`はxarray型式になるので，ラスタデータ自体は`ls_grid['cat']`でアクセスできます．

`rio.to_raster`メソッドを使って作成した災害状況ラスタをGeoTIFFで保存します．
保存するファイル名は`landslide_classification.tif`です．
また，データのタイプを16ビット整数（`dtype='int16'`）にします．

In [ ]:
# ラスター化した災害状況データをGeoTIFF形式で保存
ls_grid['cat'].rio.to_raster(base_dir + "landslide_classification.tif",
                            driver='GTiff', dtype='int16')

保存した災害状況ラスタを明示的に読み込みます．

In [ ]:
ls_data = rasterio.open(base_dir + "landslide_classification.tif")


確認のため災害状況ラスタを表示します．

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(ls_data.read(1), vmin=0, vmax=2, cmap='rainbow')
plt.title('Landslide Classification')
plt.colorbar(img, ax=ax)
plt.show()

土砂崩壊地が水色（1），洪水流到達範囲が赤（2），それ以外が紫（0）となっています．

災害状況ラスタはカテゴリーデータ（土砂崩壊地と洪水到達範囲）になっているため，
曲率を元にした斜面形状データと同様の集計をおこない，災害状況ごとの面積比を求めます．
集計メッシュへの結合まで一気にやってしまいます．

In [ ]:
# 災害状況ラスタをメッシュごとに集計する

ls_catmap = {0: 'none', 1: 'landslide', 2: 'flood'}
stats = zonal_stats(aoi_polygon, ls_data.read(1),
                    affine=ls_data.transform, categorical=True,
                    category_map=ls_catmap)
ls_cat_df = pd.DataFrame(stats).astype(float)
# 欠損値を0で埋める
ls_cat_df = ls_cat_df.fillna(0.0)
# 災害別の面積比を計算
ls_cat_df['none_ratio'] = ls_cat_df['none'] /\
    sum(ls_cat_df.loc[0, ['none', 'flood', 'landslide']])
ls_cat_df['flood_ratio'] = ls_cat_df['flood'] /\
    sum(ls_cat_df.loc[0, ['none', 'flood', 'landslide']])
ls_cat_df['landslide_ratio'] = ls_cat_df['landslide'] /\
    sum(ls_cat_df.loc[0, ['none', 'flood', 'landslide']])
# 災害別の面積比をメッシュに結合
aoi_agg_df = pd.concat([aoi_agg_df,
    ls_cat_df[['none_ratio', 'flood_ratio', 'landslide_ratio']]], axis=1)

# 集計済みメッシュの欠損値とInfを0で埋める
aoi_agg_df.fillna(0.0)
aoi_agg_df.replace([np.inf, -np.inf], 0, inplace=True)

これで機械学習のためのデータが完成しました．
`explore`メソッドを使ってインタラクティブマップを作成しデータを確認してみます．
ここでは背景画像として地理院タイルの淡色地図，陰影起伏図と，平成29年7月九州北部豪雨のオルソ画像を使っています．
これらに土砂崩壊地ポリゴンと集計メッシュの土砂崩壊面積率を重ねてみました．

タイルを多用しているので動作が遅くなるかもしれません．

In [ ]:
mapcenter = [33.416937,130.707376]

m = folium.Map(
    location=mapcenter,
    zoom_start=13
)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='淡色地図'
).add_to(m)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/hillshademap/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='陰影起伏図'
).add_to(m)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/20170705typhoon3_0713dol1/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='平成29年7月九州北部豪雨 空中写真（朝倉地区）（2017年7月13日撮影）'
).add_to(m)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/20170705typhoon3_0713dol2/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='平成29年7月九州北部豪雨 空中写真（東峰地区）（2017年7月13日撮影）'
).add_to(m)

ls_polygon[ls_polygon['cat']==1].to_crs(epsg=3857).explore(
# ls_polygon.to_crs(epsg=3857).explore(
    m=m,
    color='red',
    name='土砂崩壊地'
)

aoi_agg_df.to_crs(epsg=3857).explore(
    m=m,
    column='landslide_ratio',
    name='土砂崩壊地面積率'
)

folium.LayerControl().add_to(m)
display(m)

## 決定木による分類・予測モデルの作成

### 決定木の作成

データが揃ったので決定木を作っていきましょう．
決定木には説明変数と目的変数が必要になります．
説明変数はメッシュごとに集計した地形量，目的変数はメッシュごとの土砂崩壊地の有無です．
Pythonのscikit-learnを使った機械学習では慣例として説明変数を`X`，目的変数を`y`とします．

まずは集計メッシュから説明変数`X`を作成します．
必要な項目は地形量の記述統計量と斜面形状の面積比です．
集計メッシュ`aoi_agg_df`のデータのうち，説明変数の列名をリスト形式で用意します．

In [ ]:
exp_val = ['alt_min', 'alt_max', 'alt_mean', 'alt_std', 'alt_range',
    'slope_min', 'slope_max', 'slope_mean', 'slope_std', 'slope_range',
    'dis_min', 'dis_max', 'dis_mean', 'dis_std', 'dis_range', 'sti_min',
    'sti_max', 'sti_mean', 'sti_std', 'sti_range', 'wi_min', 'wi_max',
    'wi_mean', 'wi_std', 'wi_range', 'plc_valley_ratio', 'plc_linear_ratio',
    'plc_ridge_ratio', 'prc_convex_ratio', 'prc_linear_ratio', 'prc_concave_ratio']

`X`に集計メッシュのうち，説明変数として必要な列のみをコピーします．

In [ ]:
X =  aoi_agg_df[exp_val]

次に目的変数を作成します．
目的変数はメッシュごとの土砂崩壊の有無になるので，
土砂崩壊地面積率が一定の値以上のメッシュを「土砂崩壊あり」とします．
ここでは土砂崩壊地の面積率が0.1（10%）以上のメッシュを崩壊地ありメッシュとし，
新しい列`landslide`に1を入れ，土砂崩壊地なしのメッシュには0を入れます．

土砂崩壊地面積率の閾値については決まった値はありません．
厳しくするのであれば面積率を大きくしてもよいでしょう．

In [ ]:
aoi_agg_df.loc[aoi_agg_df['landslide_ratio'] > 0.1, 'landslide'] = 1
aoi_agg_df = aoi_agg_df.fillna(0.0)

次に目的変数`y`を作成します．
前のステップで作成した`aoi_agg_df`の`landslide`列をそのまま`y`にコピーします．
念のため型を整数に変換します．

In [ ]:
y = aoi_agg_df['landslide'].astype(int)

説明変数と目的変数が揃いました．
次は実際に決定木を作成します．

決定木の作成には説明変数と目的変数を学習（トレーニング）データと検証（テスト）データに分ける必要があります．
学習データで決定木を作成し，作成した決定木に検証データを適用して決定木の性能を評価します．
このため，学習データと検証データは独立している（重複していない）必要があります．

一般的に学習データは検証データよりも量を多くします．
ここでは全データのうち75％を学習データ，残りの25%を検証データにしてみましょう．
説明変数の学習データを`X_train`，説明変数の検証データを`X_test`，目的変数の学習データを`y_train`，目的変数の検証データを`y_test`とします．
学習データと検証データの分割には`train_test_split`を使います．
`train_test_split`は分割したい説明変数（`X`）と目的変数（`y`），検証データの割合（`test_size=0.25`），
分割用乱数のシード（`random_state=123`），
層化分割の有無（`stratify=y`）を指定します．

`train_test_split`はランダムサンプリングにより学習データと検証データを分割するため，
実行するたびに異なる学習データと検証データが生成されます．
このため，実行するたびに作成される決定木が異なり，モデルのチューニングがしにくくなります．
分割用乱数のシードを指定すると，同じシードの値を指定している限り同じ乱数が生成されるため，
同一の学習データと検証データで決定木を作成することができます．

また目的変数のクラス（値）に不均衡がある場合，学習データと検証データの中の土砂崩壊の有無の数が異なってしまうことがあります．
この状態で学習・検証を行っても正しい結果を得ることができません．
`stratify=y`として層化分割を行うことにより，元データの土砂災害の有無の比率を保ったまま学習データと検証データに分けることができます．


In [ ]:
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.25, random_state=123, stratify=y)

それではいよいよ決定木を作成します．
作成の手順は以下です．

1. 決定木の設定（ハイパーパラメーターの設定）を行う
2. 設定に基づいて学習データから決定木を作成する
3. 作成した決定木に検証データを適用する
4. 学習結果と検証結果の正解率を検討する
5. 混同行列（confusion matrix）と各種評価指標からモデルの性能を検討する
6. 決定木を作成して分類プロセスを検討する


`DecisionTreeClassifier`を使って決定木の設定を行います．
ここではシンプルに2項目のみ設定します．
`max_depth`は作成する決定木の最大の深さで，ここでは3とします．
`random_state`は乱数のシードで，同じ値を設定することにより同じ結果が得られるようにします．

In [ ]:
model = DecisionTreeClassifier(max_depth=3, random_state=123)

`model.fit`を使って学習します．
引数は学習用の説明変数と目的変数です．

In [ ]:
model.fit(X_train, y_train)

作成した決定木を検証データに適用します．
こちらは`predict`メソッドを使い，検証用説明変数を指定します．

In [ ]:
y_pred = model.predict(X_test)

以上で決定木が作成されました．
結果の検証をしてみましょう．
まずは学習データと検証データの正解率を表示します．

In [ ]:
print("学習データの正解率", model.score(X_train, y_train))
print("検証データの正解率", model.score(X_test, y_test))

学習データと検証データの両者とも正解率は約0.82となっています．
このことから，学習データと検証データの土砂崩壊地の有無は，
82%ほどが正しく分類されていることがわかります．

次は82%の正解率の内訳について，混同行列と各種評価指標から検討していきます．

混同行列（confusion matrix）は実際の値と予測値の値を行列で示したもので，
ここではメッシュごとの土砂崩壊の有無（「実際の値」とします）と
決定木をたどって予測された土砂崩壊の有無（「予測値」とします）の行列です．
目的変数が土砂崩壊の有無の2値なので，混同行列も2*2の行列になります．

混同行列は`confusion_matrix`で作成します．
引数は目的変数の学習データと目的変数の予測値です．


In [ ]:
print(confusion_matrix(y_test, y_pred))

以上のようになりました．
これは以下のように読み取れます．

|      |      |予測 |  |
| ---- | ---- | ---- | ---- |
|      |      | なし | あり |
| 実際 | なし | 2186 | 1    |
|  | あり | 477  | 0    |


* 検証用データは2186+477+1+0=2664サンプルあり，そのうち2186+1=2187サンプルが土砂崩壊なし，477+0=477サンプルが土砂崩壊ありだった．これが「実際」となる．
* 検証用データを決定木に適用した結果，土砂災害なしが2186+477=2663サンプル，土砂災害ありが1+0=1サンプルとなった．これが「予測」となる．
* 検証データの正解率は(2186+0)/(2186+477+1+0)=0.820570570570571になる．

決定木の性能を表す指標を表示してみましょう．
`classification_report`に目的変数の検証データ（`y-test`）と目的変数の予測値（`y_pred`）を与えます．

In [ ]:
print(classification_report(y_test, y_pred,
                            target_names=['LS-no', 'LS-yes'] ))

いくつか指標が示されていますが，ここではprecisionを考えてみましょう．
precisionは予測されたものと実際の比を示します．
混同行列からLS-no（土砂崩壊なし）と予測されたサンプル数は2186+477=2663サンプル，実際にLS-noだったサンプル数は2186+1=2187なので，
LS_noのprecisionは2187/2663=0.8212となります．
同様にLS-yes（土砂崩壊なし）のprecisionは(1+0)/(477+0)=0.00です．

`tree_plot`を使って決定木を表示します．
主な引数は説明変数のリスト（`feature_name=exp_val`），目的変数のクラス名（`class_names=["LS-no", "LS-yes"]`），
ノード番号（`node_ids=True`）です．
`filled=True`を指定するとクラスとgini不純度の値で塗りつぶし色が変わります．


In [ ]:
# 決定木を表示する
plt.figure(figsize=(15, 10))
tree.plot_tree(model, feature_names=exp_val,
                class_names=["LS-no", "LS-yes"], filled=True, node_ids=True)
plt.title("Decision Tree")
plt.show()

`max_depth=3`にしたので4段（最上位に加えて3段）のツリーができています．
ツリーを構成する四角をノード（node）と呼び，最末端のノードを葉ノード（leaf node）と呼びます．
各ノードにはノード番号（`node`），分岐の際に参照される説明変数名としきい値，gini不純度（`gini`），ノードに含まれるサンプル数（`sample`）とその内訳（`value`），
内訳のうち最大クラスのクラス名（`class`）が書かれています．
ノードからは下向きに2本の矢印が書かれており，しきい値を満たすと左下（`True`），満たさない場合は右下（`False`）に進みます．
ノードの色はノードのクラス（`class`）によってオレンジ（土砂崩壊なし`calss=LS_no`）と青（土砂崩壊なし`calss=LS_yes`）に塗られ，
色の濃さはgini不純度が小さい（0に近い）ほど濃くなります．

学習用データと検証用データの両者とも，メッシュごとに最上位のノードから決定木に通され，
分岐条件に従っていずれかの葉ノードにつきます．
葉ノードに着いたときに予測値として葉ノードのクラスが与えられます．

まず最上位のノードを見てみましょう．
このノードに含まれるサンプル数（`samples`）は7992個で，内訳（`value`）は土砂崩壊なしが6559，土砂崩壊ありが1433個です．
gini不純度は内訳（土砂崩壊の有無の数）から求まる指標で，内訳が偏っている場合は0に近くなり，偏りがない場合は1（内訳が2値の場合は最大で0.5）になります．
最上位ノードでは土砂崩壊なしに偏っているためにgini不純度が0.294となっています．
また，このノードの内訳（`values`）は土砂崩壊なしが多いためクラス（`class`）が`LS-no`になります．
最上位ノードの分岐条件は侵食高の最大値（`dis_max`）が34.408以下かどうかです．
条件を満たす場合（`True`）は左下の1番ノードに進み，満たさない場合（`False`）は右下の8番ノードに進みます．


決定木の全体を見てみましょう．
8個ある葉ノードのうち，7個が土砂崩壊なし（`LS-no`）でわずか1個（4番ノード）が土砂崩壊あり（`LS-yes`）です．
4番ノードの内訳（`values`）を見てみると，土砂崩壊なしが1，ありが3でサンプル数が非常に少なく，かろうじて土砂崩壊ありのノードになっています．
むしろ4番ノード以外の葉ノード（いずれも土砂崩壊なしになる）に含まれる土砂崩壊ありのサンプルの方が多いのですが，
土砂崩壊なしのサンプルの方が数が多いために埋もれてしまっています．

以上のことから，この決定木は正解率が0.82で一見よい結果を得られているように思えますが，ほとんどを土砂災害なしに分類する決定木になっているため土砂崩壊を正確に予測することができません．
これはなぜでしょうか．

一番大きな原因はもともと土砂崩壊ありのメッシュがなしのメッシュよりもかなり少ないということがあります．
学習データ全体で7992サンプルありますが，このうち土砂崩壊ありメッシュは1433サンプルでわずか18%ほどです．
つまりすべての葉ノードが土砂崩壊なしになる決定木が作られて
18%の土砂崩壊ありメッシュがすべて土砂崩壊なしに誤って予測されても，
残り82%の土砂崩壊なしが正しく予測されるため全体の正解率は0.82となります．

結果的にこの決定木は「土砂崩壊がないメッシュ」を判別するモデルになっています．



### 決定木のチューニング

#### 土砂災害ありとなしの数の違いを考慮する

作成した決定木は思った通りの性能が出なかったので，チューニングを行っていきます．

まず最初に土砂崩壊の有無に差があることを解決しましょう．
`DecisionTreeClassifier`の引数に`class_weight='balanced'`を指定します．
こうすることにより目的変数の偏りをある程度解消することができます．

決定木を実行して混同行列と分類レポートを表示します．

In [ ]:
# 土砂災害ありメッシュとなしメッシュの数の違いを考慮する
model = DecisionTreeClassifier(class_weight='balanced',
                                max_depth=3, random_state=123)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("学習データの正解率", model.score(X_train, y_train))
print("検証データの正解率", model.score(X_test, y_test))

print("混同行列")
print(confusion_matrix(y_test, y_pred))

print("分類レポート")
print(classification_report(y_test, y_pred, target_names=['LS-no', 'LS-yes'] ))


学習データ・検証データ共に正解率は0.57前後になりました．
最初に作成した決定木よりも数字は落ちています．
混同行列を見てみましょう．

2値の混同行列はこのような構造をしています．

|      |      |予測 |  |
| ---- | ---- | ---- | ---- |
|      |      | なし | あり |
| 実際 | なし | TN | FP    |
|  | あり | FN  | TP    |


TNはTrue Negative，FPはFalse Positive，FNはFalse NegativTrueはTrue Positiveの略です．
Positiveは陽性（注目している事象が発現していること．ここでは土砂崩壊が発生していることをあらわす），Negativeは陰性（注目している事象が発現していないこと，ここでは土砂崩壊が発生していないことを表す）で，Trueは真，Falseは偽です．

今回作成した決定木の混同行列をこれに当てはめてみるとTN（真に土砂崩壊なし）が1148，TP（真に土砂崩壊あり）が359になっており，この2個が正解になります．
正解率（accuracy）は(TN+TP)/(TN+FN+FP+TP)で求まるため(1148+359)/(1148+118+1039+359)=0.565690となります．
precisionは(TP)/(TP+FP)なので土砂崩壊あり（`LS_yes`）のprecisionは359/(1039+359)=2.2567954（約0.26）になります．
同様に土砂崩壊なし（`LS-no`）のprecisionは(TN)/(TN+FN)なので1148/(1148+118)=0.906793（約0.91）です．
最初に作成した決定木に比べると正解率は落ちていますが，土砂崩壊ありを判別できる決定木になっています．


決定木も表示してみましょう．

In [ ]:
# 決定木を表示する
plt.figure(figsize=(15, 10))
tree.plot_tree(model, feature_names=exp_val,
                class_names=["LS-no", "LS-yes"], filled=True, node_ids=True)
plt.title("Decision Tree")
plt.show()

土砂崩壊ありの葉ノードは相変わらず一つだけ（ノード11）です．
各ノードの分岐条件を見ていきます．
最上位ノードの分岐条件は標高の値域（`alt_range`）で閾値は18.832です．
18.832以下の場合は左下（ノード1）に進み，ノード1より下のノードはすべて土砂災害なし（`class=LS-no`）です．
つまり，標高の値域が18.832以下では土砂崩壊が起こりにくいと言えます．

ノード8以下は土砂崩壊なし（`class=LS-no`）と土砂崩壊あり（`class=LS-yes`）が混在しており，
ノード8以下でさらに細かく分かれていきます．
ノード8の分岐条件を見てみると標高の最大値（`alt_max`）が出てきており，閾値は528.699となっています．
同様にノード12の分岐条件を見てみると標高の最小値（`alt_min`）です．
一見良さそうですが，実はここに問題点が隠れています．

例えば標高が最大で1000mある山地で土砂崩壊があったとしましょう．
この場合，土砂崩壊のあるメッシュの標高の最大値や最小値が1000を超えることはありません．
このように絶対的な標高に関する地形量を入れてしまうと，対象地域の高度分布に特化した分岐条件が出てしまうことがあります．
このため，他の地域でも適用できる汎用的な決定木を作成することができなくなります．
このため説明変数を取捨選択する必要があります．


#### 説明変数の取捨選択

絶対的な標高に関する地形量を説明変数から除外します．
除外する地形量は標高の最小値（`alt_min`），標高の最大値（`alt_max`），標高の平均値（`alt_mean`）です．
標高の標準偏差（`alt_std`）と標高の値域（`alt_range`）はメッシュ内標高の相対的な量なので（厳密には標高が高くなるほど標準偏差も値域も大きくなりますが），
説明変数にくわえたままにします．

説明変数の抽出からやり直します．

In [ ]:
# 'alt_min', 'alt_max', 'alt_mean', は除外する
exp_val = ['alt_std', 'alt_range',
    'slope_min', 'slope_max', 'slope_mean', 'slope_std', 'slope_range',
    'dis_min', 'dis_max', 'dis_mean', 'dis_std', 'dis_range', 'sti_min',
    'sti_max', 'sti_mean', 'sti_std', 'sti_range', 'wi_min', 'wi_max',
    'wi_mean', 'wi_std', 'wi_range', 'plc_valley_ratio', 'plc_linear_ratio',
    'plc_ridge_ratio', 'prc_convex_ratio', 'prc_linear_ratio', 'prc_concave_ratio']

# 説明変数を再度作成する
X = aoi_agg_df[exp_val]

# 目的変数を再度作成する
y = aoi_agg_df['landslide'].astype(int)

# データをトレーニングデータと検証データに分ける
X_train, X_test, y_train, y_test = train_test_split(X, y,
                        test_size=0.25, random_state=123, stratify=y)

決定木を作成します．

In [ ]:
# 土砂災害ありメッシュとなしメッシュの数の違いを考慮する
model = DecisionTreeClassifier(class_weight='balanced',
                                max_depth=3, random_state=123)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("学習データの正解率", model.score(X_train, y_train))
print("検証データの正解率", model.score(X_test, y_test))

print("混同行列")
print(confusion_matrix(y_test, y_pred))

print("分類レポート")
print(classification_report(y_test, y_pred,
                            target_names=['LS-no', 'LS-yes'] ))

正解率が向上しました．
また，混同行列からTP（土砂崩壊ありの正解）も増えたことがわかります．

決定木も表示します．

In [ ]:
# 決定木を表示する
plt.figure(figsize=(15, 10))

tree.plot_tree(model, feature_names=exp_val,
                class_names=["LS-no", "LS-yes"], filled=True, node_ids=True)
plt.title("Decision Tree")
plt.show()

#### 決定木の枝刈り

今までの決定木の形は最上位ノードより下が3段で，葉ノードの数が8個に固定されています．
新たねパラメーターを`DecisionTreeClassifier`に加えることで決定木の形を変えることができます．
このように決定木の形を変えることを枝刈り（pluning）と呼びます．
枝刈りは不要な分岐を抑制してわかりやすい決定木を作るだけでなく，
学習データに特化した決定木ができてしまう，いわゆる過学習を抑えることができます．

枝刈り（決定木の形状の変更）に関する主なパラメーターには以下の3点があります．
これらのパラメーターのことをハイパーパラメーターと呼び，
性能のよい決定木を作るために重要な概念となります．

* `max_depth`：木の最大深さ
* `max_leaf_nodes`：葉の最大数
* `min_sample_leaf`：一つの葉を構成する最小のサンプル数

ここでは木の最大深さを20（`max_depth=20`），葉の最大数を100（`max_leaf_nodes=100`），
一つの葉を構成する最小のサンプル数を1000（`min_sample_leaf=1000`）で決定木を作ってみましょう．

In [ ]:
# ハイパーパラメータを変更して試してみてください．
# 本稿では以下のハイパーパラメーターで最終的な決定木を作ります．
# model = DecisionTreeClassifier(class_weight='balanced', max_depth=20,
#   max_leaf_nodes=100, min_samples_leaf=1000, random_state=123)

model = DecisionTreeClassifier(class_weight='balanced',
                                max_depth=20, max_leaf_nodes=100,
                                min_samples_leaf=1000, random_state=123)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("学習データの正解率", model.score(X_train, y_train))
print("検証データの正解率", model.score(X_test, y_test))

print("混同行列")
print(confusion_matrix(y_test, y_pred))

print("分類レポート")
print(classification_report(y_test, y_pred,
                            target_names=['LS-no', 'LS-yes'] ))


正解率は落ちていますがprecisionはそれほど落ちていません．
混同行列からFN（実際の土砂崩壊はあるが土砂崩壊なしと予測されたサンプル数）が少なく，
TP（実際も予測も土砂崩壊あり）が多くなっていることから，土砂崩壊地の取りこぼし（LS-yesのrecall）が少なくなり，
決定木の性能が向上しています．

このように，ハイパーパラメーターを変えることにより決定木の形が変わり，
全体精度や各種評価指標も変化します．
機械学習では最もよいハイパーパラメーターを見つけることが重要になってきます．

本講習では行いませんが，最適なハイパーパラメーターを探す方法にグリッドサーチがあります．
気になる方は調べてみてください．


決定木も表示してみましょう．

In [ ]:
# 決定木を表示する
plt.figure(figsize=(15, 10))
tree.plot_tree(model, feature_names=exp_val, class_names=["LS-no", "LS-yes"],
                filled=True, node_ids=True)
plt.title("Decision Tree")
plt.show()

全体的に決定木が小さくなりました．
葉ノードの数は6個で，土砂崩壊ありのハノードも3個になりました．
見やすい（解釈しやすい）決定木ができたので，とりあえずこれで完成とします．


### 決定木による土砂崩壊マップの作成

完成した決定木を用いて土砂崩壊マップを作成しましょう．
決定木の作成ではデータを学習用と検証用に分けましたが，
マップ作りでは全データに対して決定木を適用し，その結果を表示します．

決定木に適用するデータを作成します．
標高の絶対値に関する地形量は除外し，全メッシュを含むデータを作成します．


In [ ]:
# 'alt_min', 'alt_max', 'alt_mean', は除外する
exp_val = ['alt_std', 'alt_range',
    'slope_min', 'slope_max', 'slope_mean', 'slope_std', 'slope_range',
    'dis_min', 'dis_max', 'dis_mean', 'dis_std', 'dis_range', 'sti_min',
    'sti_max', 'sti_mean', 'sti_std', 'sti_range', 'wi_min', 'wi_max',
    'wi_mean', 'wi_std', 'wi_range', 'plc_valley_ratio', 'plc_linear_ratio',
    'plc_ridge_ratio', 'prc_convex_ratio', 'prc_linear_ratio', 'prc_concave_ratio']

# 全メッシュの説明変数を作成する
all_data = aoi_agg_df[exp_val]

作成した全メッシュのデータに対して決定木を適用し，予測値を求めます．
予測値は`all_pred_df`の`ls_pred`列に格納されます．
1は土砂崩壊あり，0は土砂崩壊なしです．

In [ ]:
# 全メッシュに対して決定木を適用し，
# 土砂崩壊の有無を予測する
all_pred = model.predict(all_data).tolist()

# 予測結果のデータフレームを作成する．
all_pred_df = pd.DataFrame(all_pred, columns=['ls_pred'])

次に`apply`メソッドを使ってメッシュごとにどのノードに属するか求めます．
結果は`all_nodes_df`の`node`列に格納されます．

In [ ]:
all_nodes=model.apply(all_data).tolist()
all_nodes_df = pd.DataFrame(all_nodes, columns=['node'])

`pd.concat`を使って全メッシュの予測値とノードを集計メッシュ（`aoi_agg_df`）に横結合します．

In [ ]:
aoi_agg_df = pd.concat([aoi_agg_df, all_pred_df, all_nodes_df], axis=1)

`aoi_agg_df`の`landslide`列は実際の土砂崩壊の有無を，`ls_pred`列は土砂崩壊の有無の予測値です．
この2列を使ってメッシュが正しく予測されてるかどうかを表すラベルをつけます．
実際に土砂崩壊があり（`landslide==1`），予測値でも土砂崩壊がある（`ls_pred==1`）メッシュは真陽性（True Positive）なのでラベル`TP`，
実際に土砂崩壊があり（`landslide==1`），予測値では土砂崩壊がない（`ls_pred==0`）メッシュは偽陰性（False Negative）なのでラベル`FN`，
実際に土砂崩壊がなく（`landslide==0`），予測値でも土砂崩壊がない（`ls_pred==0`）メッシュは真陰性（True Negative）なのでラベル`TN`，
実際に土砂崩壊がなく（`landslide==0`），予測値では土砂崩壊がある（`ls_pred==1`）メッシュは偽陽性（False Negative）なのでラベル`FN`をつけます．
このラベルは`aoi_agg_df`の`conmat`列に格納します．

In [ ]:
# 実際に土砂崩壊があり（landslide==1），予測値でもある（ls_pred==1）メッシュのconmatをTPに設定
aoi_agg_df.loc[(aoi_agg_df['landslide']==1) & (aoi_agg_df['ls_pred']==1),
                'conmat'] = 'TP'

# 実際に土砂崩壊があり（landslide==1），予測値ではない（ls_pred==0）メッシュのconmatをFNに設定
aoi_agg_df.loc[(aoi_agg_df['landslide']==1) & (aoi_agg_df['ls_pred']==0),
                'conmat'] = 'FN'

# 実際に土砂崩壊がなく（landslide==0），予測値でもない（ls_pred==0）メッシュのconmatをTNに設定
aoi_agg_df.loc[(aoi_agg_df['landslide']==0) & (aoi_agg_df['ls_pred']==0),
                'conmat'] = 'TN'

# 実際に土砂崩壊がなく（landslide==0），予測値ではある（ls_pred==1）メッシュのconmatをFPに設定
aoi_agg_df.loc[(aoi_agg_df['landslide']==0) & (aoi_agg_df['ls_pred']==1),
                'conmat'] = 'FP'

混同行列のラベルを地図にプロットしてみましょう．

In [ ]:
aoi_agg_df.plot(column='conmat', legend=True)

FPがかなり多い事に気づいたでしょうか．
FPは実際には土砂崩壊なしで予測では土砂崩壊ありとされたメッシュです．
全メッシュにおける実際の土砂崩壊の有無と予測による土砂崩壊の有無から混同行列を作ってみます．

`confusion_matrix`に実際の土砂災害の有無を格納している列（`aoi_agg_df['landslide']`）と
土砂災害の有無の予測値を格納している列（`aoi_agg_df['ls_pred']`）を指定します．

In [ ]:
print(confusion_matrix(aoi_agg_df['landslide'], aoi_agg_df['ls_pred']))


|      |      |予測 |  |
| ---- | ---- | ---- | ---- |
|      |      | なし | あり |
| 実際 | なし | 4558 |4158|
|  | あり | 507  | 1403    |


TP（実際も予測も土砂崩壊あり）が1403サンプルに対してFP（実際にはないが予測ではあり）が4158となっています．
これは実際には土砂崩壊なし（4558+4158）の半分近くです．
つまり，実際に土砂崩壊がなかったメッシュのうち半分近くは土砂崩壊が「あり」と予測されています．
かなりの過推定となっており，機械学習のモデルとしてはあまり良くないように思えます．
本来の機械学習はTPとTNが大きくなることが望ましいとされていますが，
今回の例でもそうなのでしょうか．

今回の決定木の目的は土砂崩壊があるメッシュを推定することです．
そのためにDEMから求めた地形量を説明変数，土砂崩壊の実績を目的変数として決定木を作成しました．
しかし対象事例とした平成29年7月九州北部豪雨の土砂崩壊は山地に降った大雨によってもたらされています．

一般に災害の原因は素因と誘因に分けられます．
素因は災害が起きる場の条件（今回の例では土砂崩壊が起きやすい地形），誘因は災害を直接引き起こすトリガーを示しています．
今回の分析には素因に関する説明変数のみを用いており，誘因（大雨）に関する変数は入っていません．
もしかすると平成29年7月九州北部豪雨の雨に関する変数を加えれば，もっと良い決定木を作ることが出来るかもしれません．
しかし雨の分布は地域や事例による差が大きいため，
説明変数に雨の情報を加えてしまうと特定の事例の雨（この場合は平成29年7月九州北部豪雨）が
降ったときに土砂崩壊を起こす場所を推定する決定木になってしまう恐れがあります．
これだと作成した決定木は平成29年7月九州北部豪雨の時にのみ使えるものになってしまい，
汎用性のない分類・推定モデルになってしまいます．

あえて誘因を説明変数から外すことで，素因（地形）からみて土砂崩壊のポテンシャルが高い地域を取り出せると考えることができます．
これを考慮すると，今回作成した決定木のFP（実際には土砂崩壊は起きていないが，予測上は土砂崩壊が起きるとされた場所）に相当するメッシュは，
平成29年7月九州北部豪雨ではたまたま土砂崩壊が起きていないが，別の大雨が降った時には土砂崩壊が起こりえる場所と考えることができます．
つまりTPとFPのメッシュは「土砂崩壊のポテンシャルが高い場所」と解釈できるのではないでしょうか．

メッシュがどのノードに属するかを表すマップも表示してみます．

In [ ]:
aoi_agg_df.plot(column='node', categorical=True, legend=True)

決定木の葉ノードのノード番号ごとに分類された地図です．
同じノードのメッシュは地形が似ていると考えられます．
このマップを詳しく見ることにより，なぜ土砂崩壊が起きるのか，もしくは起きないのかを考察することができます．

決定木から，ノード1，5，6は土砂崩壊なし，ノード7，9，10は土砂崩壊ありです．
今回使用した土砂崩壊値のポリゴンは，崩壊発生域，土砂移動域，土砂堆積域の3つの状態がすべて合わさって
土砂崩壊ポリゴンになっています．
決定木の土砂崩壊ありの葉ノードにたどり着くまでの分岐条件を詳しく見ていくことで，
ある土砂崩壊ありの葉ノードは崩壊発生域，別の崩壊ありの葉ノードは土砂移動域，
また別の葉ノードは土砂堆積域を表しているという考察を得られるかもしれません．
つまりノードごとに土砂崩壊の状態が異なることがわかるかもしれません．
これはマップに表すことで補強することができます．

このように土砂崩壊を分類する決定木を作り，混同行列のマップを作ることにより土砂崩壊ポテンシャルの空間分布がわかり，
葉ノードのマップを作ることにより土砂崩壊の状態の空間分布がわかってきます．

## 結果のエクスポート

最後に完成した集計データをGISで使うためにエクスポートします．

まずはGeoJSONに書き出します．
ArcGISでGeoJSONを読むためには，ツールボックスを使ってジオデータベースに変換してから読み込んでください．
QGISは直接GeoJSONを扱うことができます．

`aoi_agg_df`の`to_file`メソッドでGeoJSONを指定します．
実行するとデータフォルダ（Google Colaboratoryを使っている場合はGoogle Driveの`gisday2025`フォルダ）に
`aoi_aggregated_data.geojson`というファイル名で保存されます．

In [ ]:
aoi_agg_df.to_file(base_dir + "aoi_aggregated_data.geojson",
                    driver='GeoJSON')

シェープファイルにもエクスポートしましょう．
こちらも`to_file`メソッドでシェープファイルを指定します．
保存先はGeoJSONの場合と同様です．

In [ ]:
aoi_agg_df.to_file(base_dir + "aoi_aggregated_data.shp",
                    driver='ESRI Shapefile')

## 集計結果をインタラクティブマップ表示する

最後におまけです．
結果をインタラクティブマップに表示してみましょう．
データが多いので重たいかもしれません．

In [ ]:
mapcenter = [33.416937,130.707376]

m = folium.Map(
    location=mapcenter,
    zoom_start=13
)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='淡色地図'
).add_to(m)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/hillshademap/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='陰影起伏図'
).add_to(m)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/20170705typhoon3_0713dol1/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='平成29年7月九州北部豪雨 空中写真（朝倉地区）（2017年7月13日撮影）'
).add_to(m)

folium.raster_layers.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/20170705typhoon3_0713dol2/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='平成29年7月九州北部豪雨 空中写真（東峰地区）（2017年7月13日撮影）'
).add_to(m)

ls_polygon[ls_polygon['cat']==1].to_crs(epsg=3857).explore(
    m=m,
    color='red',
    name='土砂崩壊地'
)

aoi_agg_df.to_crs(epsg=3857).explore(
    m=m,
    column='conmat',
    categorical=True,
    name='混同行列'
)

aoi_agg_df.to_crs(epsg=3857).explore(
    m=m,
    column='node',
    categorical=True,
    name='ノード'
)

folium.LayerControl().add_to(m)
display(m)